In [1]:
from pathlib import Path
from utils import get_project_root, read_json
from utils.graph_untils import load_entities

entity2id = load_entities()
default_config = read_json(Path(get_project_root()) / "config" / "mind_rs_default.json")

In [2]:
from collections import defaultdict
import json


def load_news_entity_from_file(news_file):
    news_entity = defaultdict()
    with open(news_file, "r", encoding="utf-8") as rd:
        for text in rd:
            # news id, category, subcategory, title, abstract, url
            nid, vert, subvert, title, abstract, url, title_entity, abs_entity = text.strip("\n").split("\t")
            title_entity_json, abstract_entity_json = json.loads(title_entity), json.loads(abs_entity)
            for entity in title_entity_json+abstract_entity_json:
                news_entity[entity["WikidataId"]] = entity["SurfaceForms"]
    return news_entity

In [10]:
root_path = Path(default_config["data_config"]["data_path"])
news_entities = defaultdict()
phases = ["train", "valid", "test"]
mind_type = "large"
for phase in phases:
    file = root_path / mind_type / phase / "news.tsv"
    if file.exists():
        news_entities.update(load_news_entity_from_file(file))

In [4]:
from utils.graph_untils import kg_default_root

entity2form = defaultdict()
description_file = kg_default_root() / "description.txt"
with open(description_file, "r", encoding="utf-8") as rd:
    for line in rd.readlines():
        entity_id = line.split()[0]
        if entity_id not in news_entities.keys():
            continue
        entity2form[entity_id] = line.replace(entity_id, "").split()

In [6]:
entity2label = defaultdict()
label_file = kg_default_root() / "label.txt"
with open(label_file, "r", encoding="utf-8") as rd:
    for line in rd.readlines():
        entity_id = line.split()[0]
        if entity_id not in news_entities.keys():
            continue
        entity2label[entity_id] = line.replace(entity_id, "").split()

In [9]:
for eid, form in news_entities.items():
    if eid in entity2label:
        if form != entity2label[eid]:
            print(eid, form, entity2label[eid])

Q80976 ['Prince Philip'] ['Prince', 'Philip,', 'Duke', 'of', 'Edinburgh', 'Prince', 'Philip', 'of', 'Greece', 'and', 'Denmark', 'Philip', 'Mountbatten', 'Prince', 'Philip', 'Duke', 'of', 'Edinburgh', 'Prince', 'Phillip']
Q43274 ['Prince Charles'] ['Charles,', 'Prince', 'of', 'Wales', 'Charles', 'Philip', 'Arthur', 'George', 'The', 'Duke', 'of', 'Rothesay', 'The', 'Duke', 'of', 'Cornwall', 'Prince', 'of', 'Wales', 'Charles', 'Mountbatten-Windsor', 'Charles', 'Philip', 'Arthur', 'George', 'Mountbatten-Windsor', 'Prince', 'Charles', 'Prince', 'of', 'Wales', 'Charles', 'Charles', 'Wales', 'Prince', 'of', 'Wales', 'and', 'Duke', 'of', 'Rothesay', 'Charles', 'Philip', 'Arthur', 'George', 'HRH', 'The', 'Prince', 'of', 'Wales', 'HRH', 'Prince', 'Charles,', 'The', 'Prince', 'of', 'Wales', 'Prince', 'Charles,', 'The', 'Prince', 'of', 'Wales', 'Charles', 'Windsor', 'Charles', 'Windsor,', 'Prince', 'of', 'Wales', 'The', 'Prince', 'Of', 'Wales', 'HRH', 'Prince', 'Charles', 'HRH', 'The', 'Prince', '